In [0]:
## Jupyter Notebook PySpark with Python3 on Azure Databricks
## Collect data from insideairbnb.com by HTTP requests; This data is licensed under a Creative Commons Attribution 4.0 International License: https://creativecommons.org/licenses/by/4.0/legalcode

In [0]:
%sh
 pip install beautifulsoup4

Collecting beautifulsoup4
 Downloading beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
Collecting soupsieve>1.2
 Downloading soupsieve-2.3.2.post1-py3-none-any.whl (37 kB)
Installing collected packages: soupsieve, beautifulsoup4
Successfully installed beautifulsoup4-4.11.1 soupsieve-2.3.2.post1
WARNING: You are using pip version 21.0.1; however, version 22.1.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
%sh
pip install html5lib

Collecting html5lib
 Downloading html5lib-1.1-py2.py3-none-any.whl (112 kB)
Requirement already satisfied: six>=1.9 in /databricks/python3/lib/python3.8/site-packages (from html5lib) (1.15.0)
Requirement already satisfied: webencodings in /databricks/python3/lib/python3.8/site-packages (from html5lib) (0.5.1)
Installing collected packages: html5lib
Successfully installed html5lib-1.1
WARNING: You are using pip version 21.0.1; however, version 22.1.2 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
import sys
import os
import requests
from bs4 import BeautifulSoup

In [0]:
storageAccountName = 'strfactsblob'
storageAccountAccessKey = 'TpOQSdCGOmT7XjOC+47zvsS5ba+mm8CUJHtp8aia6oBgUGVQGqP8mKm4ShVe4U72ZdWvyl9kRCeW+AStY9WK8Q=='
ContainerName = 'bcontainer1'
 
if not any(mount.mountPoint == '/mnt/blob1/' for mount in dbutils.fs.mounts()):
    try:
        dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(ContainerName, storageAccountName),
        mount_point = "/mnt/blob1",
        extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
        )
    except Exception as e:
        print('mount exception')
display(dbutils.fs.ls('mnt/blob1'))

mount exception

path,name,size,modificationTime
dbfs:/mnt/blob1/indata/,indata/,0,0


In [0]:
#indata = 'c:/sb/strfacts/indata'
indata = '/mnt/blob1/indata'

In [0]:
fns = ("listings.csv.gz", "calendar.csv.gz", "reviews.csv.gz", "listings.csv", "reviews.csv", "neighbourhoods.csv", "neighbourhoods.geojson")

In [0]:
url = "http://insideairbnb.com/get-the-data.html"
rmain = requests.get(url)
#soup = BeautifulSoup(rmain.content, 'html5lib')
soup = BeautifulSoup(rmain.content, 'html.parser')
#print(soup.prettify())
#loop for all links
for link in soup.find_all('a'):
    href = link.get('href')
    if href is None:
        continue    #skip to the next loop
    temp = href.split('/')
    #temp[-1] - calendar.csv.gz
    #temp[-3] - 2021-12-05
    #temp[-4] - hawaii
    if temp[-1] in fns: 
        #ym - yyyy-mm
        ym = temp[-3][0:7]
        #test if folder already exists
        #dn = indata+'/'+ym
        dn = '/dbfs'+indata+'/'+ym
        if not os.path.exists(dn):
            os.makedirs(dn)
        fn = dn+'/'+temp[-4]+'-'+temp[-1]
        #test if file exists, skip to the next file
        if os.path.exists(fn):
            continue
        print(fn)
        r = requests.get(href, stream = True)   #download file
        with open(fn, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)  #save file

/dbfs/mnt/blob1/indata/2022-03/amsterdam-listings.csv.gz
/dbfs/mnt/blob1/indata/2022-03/amsterdam-calendar.csv.gz
/dbfs/mnt/blob1/indata/2022-03/amsterdam-reviews.csv.gz
/dbfs/mnt/blob1/indata/2022-03/amsterdam-listings.csv
/dbfs/mnt/blob1/indata/2022-03/amsterdam-reviews.csv
/dbfs/mnt/blob1/indata/2022-03/amsterdam-neighbourhoods.csv
/dbfs/mnt/blob1/indata/2022-03/amsterdam-neighbourhoods.geojson
/dbfs/mnt/blob1/indata/2022-03/antwerp-listings.csv.gz
/dbfs/mnt/blob1/indata/2022-03/antwerp-calendar.csv.gz
/dbfs/mnt/blob1/indata/2022-03/antwerp-reviews.csv.gz
/dbfs/mnt/blob1/indata/2022-03/antwerp-listings.csv
/dbfs/mnt/blob1/indata/2022-03/antwerp-reviews.csv
/dbfs/mnt/blob1/indata/2022-03/antwerp-neighbourhoods.csv
/dbfs/mnt/blob1/indata/2022-03/antwerp-neighbourhoods.geojson
/dbfs/mnt/blob1/indata/2022-03/asheville-listings.csv.gz
/dbfs/mnt/blob1/indata/2022-03/asheville-calendar.csv.gz
/dbfs/mnt/blob1/indata/2022-03/asheville-reviews.csv.gz
/dbfs/mnt/blob1/indata/2022-03/asheville-listings.csv
/dbfs/mnt/blob1/indata/2022-03/asheville-reviews.csv
/dbfs/mnt/blob1/indata/2022-03/asheville-neighbourhoods.csv
/dbfs/mnt/blob1/indata/2022-03/asheville-neighbourhoods.geojson
/dbfs/mnt/blob1/indata/2022-03/athens-listings.csv.gz
/dbfs/mnt/blob1/indata/2022-03/athens-calendar.csv.gz
/dbfs/mnt/blob1/indata/2022-03/athens-reviews.csv.gz
/dbfs/mnt/blob1/indata/2022-03/athens-listings.csv
/dbfs/mnt/blob1/indata/2022-03/athens-reviews.csv
/dbfs/mnt/blob1/indata/2022-03/athens-neighbourhoods.csv
/dbfs/mnt/blob1/indata/2022-03/athens-neighbourhoods.geojson
/dbfs/mnt/blob1/indata/2022-03/austin-listings.csv.gz
/dbfs/mnt/blob1/indata/2022-03/austin-calendar.csv.gz
/dbfs/mnt/blob1/indata/2022-03/austin-reviews.csv.gz
/dbfs/mnt/blob1/indata/2022-03/austin-listings.csv
/dbfs/mnt/blob1/indata/2022-03/austin-reviews.csv
/dbfs/mnt/blob1/indata/2022-03/austin-neighbourhoods.csv
/dbfs/mnt/blob1/indata/2022-03/austin-neighbourhoods.geojson
/dbfs/mnt/blob1/indata/2022-03/bangkok-listings.csv.gz
/dbfs/mnt/blob1/indata/2022-03/bangkok-calendar.csv.gz
/dbfs/mnt/blob1/indata/2022-03/bangkok-reviews.csv.gz
/dbfs/mnt/blob1/indata/2022-03/bangkok-listings.csv
/dbfs/mnt/blob1/indata/2022-03/bangkok-reviews.csv
/dbfs/mnt/blob1/indata/2022-03/bangkok-neighbourhoods.csv
/dbfs/mnt/blob1/indata/2022-03/bangkok-neighbourhoods.geojson
/dbfs/mnt/blob1/indata/2022-03/barcelona-listings.csv.gz
/dbfs/mnt/blob1/indata/2022-03/barcelona-calendar.csv.gz
/dbfs/mnt/blob1/indata/2022-03/barcelona-reviews.csv.gz
/dbfs/mnt/blob1/indata/2022-03/barcelona-listings.csv
/dbfs/mnt/blob1/indata/2022-03/barcelona-reviews.csv
/dbfs/mnt/blob1/indata/2022-03/barcelona-neighbourhoods.csv
/dbfs/mnt/blob1/indata/2022-03/barcelona-neighbourhoods.geojson
/dbfs/mnt/blob1/indata/2022-03/barossa-valley-listings.csv.gz
/dbfs/mnt/blob1/indata/2022-03/barossa-valley-calendar.csv.gz
/dbfs/mnt/blob1/indata/2022-03/barossa-valley-reviews.csv.gz
/dbfs/mnt/blob1/indata/2022-03/barossa-valley-listings.csv
/dbfs/mnt/blob1/indata/2022-03/barossa-valley-reviews.csv
/dbfs/mnt/blob1/indata/2022-03/barossa-valley-neighbourhoods.csv
/dbfs/mnt/blob1/indata/2022-03/barossa-valley-neighbourhoods.geojson
/dbfs/mnt/blob1/indata/2022-03/barwon-south-west-vic-listings.csv.gz
/dbfs/mnt/blob1/indata/2022-03/barwon-south-west-vic-calendar.csv.gz
/dbfs/mnt/blob1/indata/2022-03/barwon-south-west-vic-reviews.csv.gz
/dbfs/mnt/blob1/indata/2022-03/barwon-south-west-vic-listings.csv
/dbfs/mnt/blob1/indata/2022-03/barwon-south-west-vic-reviews.csv
/dbfs/mnt/blob1/indata/2022-03/barwon-south-west-vic-neighbourhoods.csv
/dbfs/mnt/blob1/indata/2022-03/barwon-south-west-vic-neighbourhoods.geojson
/dbfs/mnt/blob1/indata/2022-03/beijing-listings.csv.gz
/dbfs/mnt/blob1/indata/2022-03/beijing-calendar.csv.gz
/dbfs/mnt/blob1/indata/2022-03/beijing-reviews.csv.gz
/dbfs/mnt/blob1/indata/2022-03/beijing-listings.csv
/dbfs/mnt/blob1/indata/2022-03/beijing-reviews.csv
/dbfs/mnt/blob1/indata/2022-03/beijing-neighbourhoods.csv
/dbfs/mnt/blob1/i